In [1]:
import pandas as pd
import numpy as np
from cobra.io import read_sbml_model
from tqdm.auto import tqdm

In [2]:
from os import listdir
from os.path import isfile, join

mypath = '../models/non-ec/'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

['Bacteroides_uniformis_ATCC_8492.xml', 'Coprococcus_comes_ATCC_27758.xml', 'Fusobacterium_nucleatum_subsp_nucleatum_ATCC_25586.xml', 'Streptococcus_salivarius_DSM_20560.xml', 'Streptococcus_parasanguinis_ATCC_15912.xml', 'Roseburia_intestinalis_L1_82.xml', 'Escherichia_coli_ED1a.xml', 'Bacteroides_thetaiotaomicron_VPI_5482.xml']


In [3]:
db_l = []

for file in tqdm(onlyfiles):
    file = mypath + file
    model = read_sbml_model(file)
    name = model.id
    name = str(name)
    name = name.strip('M_')
    id_n = name
    name = name.replace('_',' ')
    rxns = len(model.reactions)
    mets = len(model.metabolites)
    name_list = name.split(' ')
    genus = name_list[0]
    species = ' '.join(name_list[1::])
    for i in range(1,7):
        sam_n = i
        sample = f'sample_{sam_n}'
        abundace = np.random.randint(100,500)
        res = [id_n, genus, species, rxns, mets, file, sample, abundace]
        db_l.append(res)
    
db = pd.DataFrame(db_l, columns = ['id','genus','species','reactions','metabolites','file','sample_id','abundace'])
db

  0%|          | 0/8 [00:00<?, ?it/s]

Set parameter Username


,id,genus,species,reactions,metabolites,file,sample_id,abundace
0,Bacteroides_uniformis_ATCC_8492,Bacteroides,uniformis ATCC 8492,2686,2204,../models/non-ec/Bacteroides_uniformis_ATCC_84...,sample_1,289
1,Bacteroides_uniformis_ATCC_8492,Bacteroides,uniformis ATCC 8492,2686,2204,../models/non-ec/Bacteroides_uniformis_ATCC_84...,sample_2,441
2,Bacteroides_uniformis_ATCC_8492,Bacteroides,uniformis ATCC 8492,2686,2204,../models/non-ec/Bacteroides_uniformis_ATCC_84...,sample_3,198
3,Bacteroides_uniformis_ATCC_8492,Bacteroides,uniformis ATCC 8492,2686,2204,../models/non-ec/Bacteroides_uniformis_ATCC_84...,sample_4,157
4,Bacteroides_uniformis_ATCC_8492,Bacteroides,uniformis ATCC 8492,2686,2204,../models/non-ec/Bacteroides_uniformis_ATCC_84...,sample_5,357
5,Bacteroides_uniformis_ATCC_8492,Bacteroides,uniformis ATCC 8492,2686,2204,../models/non-ec/Bacteroides_uniformis_ATCC_84...,sample_6,386
6,Coprococcus_comes_ATCC_27758,Coprococcus,comes ATCC 27758,1034,984,../models/non-ec/Coprococcus_comes_ATCC_27758.xml,sample_1,309
7,Coprococcus_comes_ATCC_27758,Coprococcus,comes ATCC 27758,1034,984,../models/non-ec/Coprococcus_comes_ATCC_27758.xml,sample_2,461
8,Coprococcus_comes_ATCC_27758,Coprococcus,comes ATCC 27758,1034,984,../models/non-ec/Coprococcus_comes_ATCC_27758.xml,sample_3,220
9,Coprococcus_comes_ATCC_27758,Coprococcus,comes ATCC 27758,1034,984,../models/non-ec/Coprococcus_comes_ATCC_27758.xml,sample_4,368


In [4]:
from micom.workflows import build

manifest = build(db, out_folder="../data/micom", model_db=None, cutoff=0.0001, threads=2)

Output()

Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username
Set parameter Username


In [5]:
manifest

,sample_id,file
0,sample_1,sample_1.pickle
1,sample_2,sample_2.pickle
2,sample_3,sample_3.pickle
3,sample_4,sample_4.pickle
4,sample_5,sample_5.pickle
5,sample_6,sample_6.pickle


In [6]:
from micom import load_pickle

com1 = load_pickle("../data/micom/sample_1.pickle")
print(len(com1.reactions))

15821


In [7]:
from micom.media import minimal_medium

med = minimal_medium(com1, 0.8)
res_m = []

for k in med.keys():
    print(f'Reaction: {k}, Flux: {med[k]}')
    rx = k
    flux = med[k]
    met = k.strip('EX_')
    res = [k,k,flux,met]
    res_m.append(res)
    
min_med = pd.DataFrame(res_m, columns = ['reaction','reaction','flux','metabolite'])
min_med

Reaction: EX_adn_m, Flux: 0.11583999999955126
Reaction: EX_ca2_m, Flux: 0.002477199999999513
Reaction: EX_cl_m, Flux: 0.002477199999999513
Reaction: EX_cobalt2_m, Flux: 0.0024772000000012895
Reaction: EX_cu2_m, Flux: 0.0024772000000012895
Reaction: EX_fe2_m, Flux: 0.004954400000000803
Reaction: EX_fe3_m, Flux: 0.0024772000000012895
Reaction: EX_gsn_m, Flux: 0.13353080000000261
Reaction: EX_k_m, Flux: 0.0024772000000012895
Reaction: EX_mg2_m, Flux: 0.0024772000000012895
Reaction: EX_mn2_m, Flux: 0.002477199999999513
Reaction: EX_pullulan1200_m, Flux: 0.0063421950979254405
Reaction: EX_pydxn_m, Flux: 0.0024772000000012895
Reaction: EX_zn2_m, Flux: 0.0024772000000012895
Reaction: EX_cgly_m, Flux: 0.06834839999999076
Reaction: EX_metala_m, Flux: 0.10471720000000001
Reaction: EX_pheme_m, Flux: 0.0024772000000012895
Reaction: EX_q8_m, Flux: 0.0024772000000012895
Reaction: EX_nmn_m, Flux: 0.364895799999303
Reaction: EX_man6p_m, Flux: 0.3648957999995872
Reaction: EX_hspg_m, Flux: 1.23854185000

,reaction,reaction,flux,metabolite
0,EX_adn_m,EX_adn_m,0.115840,adn_m
1,EX_ca2_m,EX_ca2_m,0.002477,ca2_m
2,EX_cl_m,EX_cl_m,0.002477,cl_m
3,EX_cobalt2_m,EX_cobalt2_m,0.002477,cobalt2_m
4,EX_cu2_m,EX_cu2_m,0.002477,cu2_m
5,EX_fe2_m,EX_fe2_m,0.004954,fe2_m
6,EX_fe3_m,EX_fe3_m,0.002477,fe3_m
7,EX_gsn_m,EX_gsn_m,0.133531,gsn_m
8,EX_k_m,EX_k_m,0.002477,k_m
9,EX_mg2_m,EX_mg2_m,0.002477,mg2_m


In [8]:
from micom.data import test_medium
from micom.qiime_formats import load_qiime_medium

test_med = load_qiime_medium(test_medium)
test_med

,reaction,flux,metabolite
reaction,,,
EX_glc__D_m,EX_glc__D_m,10.000000,glc__D_m
EX_nh4_m,EX_nh4_m,4.362240,nh4_m
EX_o2_m,EX_o2_m,18.579253,o2_m
EX_pi_m,EX_pi_m,2.942960,pi_m


In [14]:
from micom.workflows import grow

res = grow(manifest, model_folder="../data/micom", medium=test_med, tradeoff=1.0, threads=2)

Output()

Set parameter Username
Set parameter Username
[14:56:25] WARNING  There does not seem to be any carbon source ]8;id=317485;file:///usr/local/Caskroom/miniforge/base/envs/tmcom/lib/python3.9/site-packages/micom/community.py\community.py]8;;\:]8;id=386725;file:///usr/local/Caskroom/miniforge/base/envs/tmcom/lib/python3.9/site-packages/micom/community.py#678\678]8;;\
                    in your medium. Please double-check your                    
                    medium IDs.                                                 
[14:56:26] WARNING  There does not seem to be any carbon source ]8;id=932196;file:///usr/local/Caskroom/miniforge/base/envs/tmcom/lib/python3.9/site-packages/micom/community.py\community.py]8;;\:]8;id=510690;file:///usr/local/Caskroom/miniforge/base/envs/tmcom/lib/python3.9/site-packages/micom/community.py#678\678]8;;\
                    in your medium. Please double-check your                    
                    medium IDs.                      

In [ ]:
from micom.workflows import tradeoff

tradeoff_rates = tradeoff(manifest, model_folder="../data/micom", medium=min_med, threads=2)
tradeoff_rates.groupby("tradeoff").apply(
    lambda df: (df.growth_rate > 1e-6).sum()).reset_index()

In [ ]:
from micom.viz import plot_tradeoff

pl = plot_tradeoff(tradeoff_rates, filename="../data/micomtradeoff.html")

In [15]:
from micom.viz import plot_growth

pl = plot_growth(res, filename="../data/micomgrowth_rates.html")

In [16]:
pl.view()